In [16]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import sign

def xi_generate(x_range,e):
    bin=len(x_range)
    C=np.random.random((bin,bin))
    for i in range(bin):
        for j in range(bin):
            C[i,j]=abs(x_range[i]-x_range[j])/e
    return C

def bisection(fun, a, b, stepmax, tol):
    
    if sign(fun(a)) == 0: return a
    if sign(fun(b)) == 0: return b
        
    if sign(fun(a)) * sign(fun(b)) > 0: return print("bad section")
        
    for step in range(1, stepmax+1):
    
        c = (a + b) / 2
            
        if sign(fun(c)) == 0: return c
        
        if abs((b - a)/2) < tol: return (b + a) / 2
            
        if sign(fun(c)) * sign(fun(a)) < 0:
            b = c
        else:
            a = c
    
    return (a + b) / 2                 

n: the number of points.

In [8]:
n=4

In [9]:
dist=dict()
dist['s']=np.array([0.6,0.4])
dist['x_0']= (1/n)*np.ones(n)
dist['x_1']= np.concatenate(((1/(n-2))*np.ones(n-2),np.zeros(2)),axis=None)
dist['x']=dist['s'][0]*dist['x_0']+dist['s'][1]*dist['x_1']
dist['t_x']= (1/n)*np.ones(n)

In [10]:
v=(dist['x_0']-dist['x_1'])/dist['x']

In [11]:
e=0.01

sum(gamma) == gamma.T \mathbb{1}

sum(gamma.T) == gamma \mathbb{1}

In [12]:
xi=xi_generate([*range(n)],e)

In [13]:
# simplist
gamma=xi
for repeat in range(100):
    new_gamma=np.diag(dist['x']/sum(gamma.T))@gamma
    new2_gamma=new_gamma@np.diag(dist['t_x']/sum(new_gamma))
    gamma=new2_gamma

In [14]:
theta=abs(gamma.T @ v)*(1-0.3)

In [87]:
I=np.where(~(v==0))[0].tolist() # \overline{\supp(X)}
bbm1=np.matrix(np.ones(n)).T
px=np.matrix(dist['x']).T
ptx=np.matrix(dist['t_x']).T
V=np.matrix(v).T
Ve=np.multiply(V,np.exp(V))
#theta=np.matrix(abs(gamma.T @ v)*(1-0.3)).T
gamma_dict=dict()
gamma_dict[0]=np.matrix(xi+0.01)
gamma_dict[1]=np.matrix(np.diag((px/(gamma_dict[0] @ bbm1)).A1))@gamma_dict[0]
gamma_dict[2]=gamma_dict[1]@np.matrix(np.diag((ptx/(gamma_dict[1].T @ bbm1)).A1))
# step 3
J=np.where(~((np.matrix(gamma_dict[2]).T @ np.matrix(v).T).A1 ==0))[0].tolist()
nu=np.zeros(n)
gamma_dict[3]=np.copy(gamma_dict[2])
for j in J:
    fun = lambda z: sum(gamma_dict[2].item(i,j)*V.item(i)*np.exp(z*V.item(i)) for i in I)
    nu = bisection(fun, -5, 5, stepmax = 25, tol = 1.0e-3)
    for i in I:
        gamma_dict[3][i,j]=np.exp(nu*V.item(i))*gamma_dict[2].item(i,j)
gamma_dict[3]=np.matrix(gamma_dict[3])

#=========================
L=3
q_dict=dict()
for loop in range(1,5):
    tmp,q_dict[(loop-1)*L+1]=tmp_generator(gamma_dict,loop*L+1,q_dict,(loop-2)*L+1,L) #np.matrix(gamma_dict[3].A1*gamma_dict[0].A1/gamma_dict[1].A1)
    gamma_dict[loop*L+1]=np.matrix(np.diag((px/(tmp @ bbm1)).A1))@tmp

    tmp,q_dict[(loop-1)*L+2]=tmp_generator(gamma_dict,loop*L+2,q_dict,(loop-2)*L+2,L)  #np.matrix(gamma_dict[4].A1*gamma_dict[1].A1/gamma_dict[2].A1)
    gamma_dict[loop*L+2]=tmp@np.matrix(np.diag((ptx/(tmp.T @ bbm1)).A1))

    # step 3
    tmp,q_dict[(loop-1)*L+3]=tmp_generator(gamma_dict,loop*L+3,q_dict,(loop-2)*L+3,L)  #np.matrix(gamma_dict[5].A1*gamma_dict[2].A1/gamma_dict[3].A1)
    J=np.where(~((np.matrix(tmp).T @ np.matrix(v).T).A1 ==0))[0].tolist()
    gamma_dict[loop*L+3]=np.copy(tmp)
    for j in J:
        fun = lambda z: sum(tmp.item(i,j)*V.item(i)*np.exp(z*V.item(i)) for i in I)
        nu = bisection(fun, -5, 5, stepmax = 25, tol = 1.0e-3)
        for i in I:
            gamma_dict[loop*L+3][i,j]=np.exp(nu*V.item(i))*tmp.item(i,j)
    gamma_dict[loop*L+3]=np.matrix(gamma_dict[loop*L+3])



In [86]:
q_dict

{1: matrix([[1714.4       , 1714.4       , 1714.4       , 1714.4       ],
         [1142.97142857, 1142.97142857, 1142.97142857, 1142.97142857],
         [2666.93333333, 2666.93333333, 2666.93333333, 2666.93333333],
         [4000.26666667, 4000.26666667, 4000.26666667, 4000.26666667]]),
 2: matrix([[1506.2934775 , 1506.2934775 , 1506.2934775 , 1506.2934775 ],
         [ 913.29201   ,  913.29201   ,  913.29201   ,  913.29201   ],
         [2142.68316439, 2142.68316439, 2142.68316439, 2142.68316439],
         [3496.93600365, 3496.93600365, 3496.93600365, 3496.93600365]]),
 3: matrix([[1522.2190931 , 1443.51540042, 1581.98123502, 1550.96517121],
         [ 900.69532232,  854.12643604,  936.05651437,  917.70434435],
         [2011.4154033 , 2087.25615797, 2108.68972438, 2236.11495223],
         [3381.77726829, 3509.2877466 , 3545.32383717, 3759.56289402]]),
 4: matrix([[1504.08574814, 1426.31960855, 1563.13597708, 1532.4893903 ],
         [ 911.95342518,  864.80245826,  947.75660902,  929

In [83]:
def tmp_generator(gamma_dict,num,q_dict,q_num,L):
    if q_num<=0:
        q=np.matrix(np.ones((4,4)))
    else:
        q=q_dict[q_num]
    tmp_gamma=np.zeros((4,4))
    tmp_q=np.zeros((4,4))
    for i in range(4):
        for j in range(4):
            tmp_gamma[i,j]=q.item(i,j)*gamma_dict[num-1].item(i,j)*gamma_dict[num-L-1].item(i,j)/gamma_dict[num-L].item(i,j)
            tmp_q[i,j]=q.item(i,j)*gamma_dict[num-L-1].item(i,j)/gamma_dict[num-L].item(i,j)
    return np.matrix(tmp_gamma),np.matrix(tmp_q) 

In [81]:
gamma_dict[6]

matrix([[1.06033494e-05, 1.74944338e-01, 9.59000950e-02, 8.33645837e-02],
        [1.74898642e-01, 2.88507031e-05, 7.90880180e-02, 9.16637250e-02],
        [3.91003549e-02, 3.36914379e-02, 1.22328162e-05, 7.49257570e-02],
        [3.59364352e-02, 4.12855526e-02, 7.49618250e-02, 4.59047879e-06]])

In [64]:
gamma_dict[3].A1*gamma_dict[0].A1

array([9.30258207e-08, 1.45562000e+01, 1.74886122e+01, 2.23564393e+01,
       1.39561988e+01, 2.18292062e-07, 6.55866678e+00, 1.49042929e+01,
       5.98887463e+00, 2.67762507e+00, 9.82087479e-08, 6.37939852e+00,
       8.98331194e+00, 7.13985754e+00, 6.54877764e+00, 4.25222361e-08])

In [65]:
gamma_dict[1].A1

array([5.83294447e-06, 5.83352776e-02, 1.16664722e-01, 1.74994167e-01,
       8.75000000e-02, 8.74912509e-06, 8.75000000e-02, 1.74991251e-01,
       7.49962504e-02, 3.75000000e-02, 3.74962504e-06, 3.75000000e-02,
       7.49975002e-02, 4.99991667e-02, 2.50008333e-02, 2.49983334e-06])

In [50]:
np.matrix(np.diag((px/(tmp @ bbm1)).A1))

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 16)

In [46]:
gamma_dict[2]

matrix([[6.13995232e-06, 9.99965721e-02, 1.27269141e-01, 1.12902983e-01],
        [9.21054247e-02, 1.49974861e-05, 9.54534464e-02, 1.12901101e-01],
        [7.89435599e-02, 6.42813681e-02, 4.09045294e-06, 2.41943028e-02],
        [7.89448755e-02, 8.57070624e-02, 2.72733223e-02, 1.61284599e-06]])

In [44]:
gamma_dict[3]

array([[6.13995232e-06, 9.99965721e-02, 1.27269141e-01, 1.12902983e-01],
       [9.21054247e-02, 1.49974861e-05, 9.54534464e-02, 1.12901101e-01],
       [7.89435599e-02, 6.42813681e-02, 4.09045294e-06, 2.41943028e-02],
       [7.89448755e-02, 8.57070624e-02, 2.72733223e-02, 1.61284599e-06]])

In [31]:
np.matrix(gamma_dict[1].A1*gamma_dict[0].A1/gamma_dict[2].A1)

matrix([[9.49998334e-03, 5.83431111e+01, 1.83344611e+02, 4.65001001e+02,
         9.50093334e+01, 5.83372774e-03, 9.16768889e+01, 3.10005834e+02,
         1.90009167e+02, 5.83431111e+01, 9.16677221e-03, 1.55010667e+02,
         2.85009000e+02, 1.16680389e+02, 9.16768889e+01, 1.54995167e-02]])

In [25]:
gamma_dict[3].T@ V

matrix([[ 0.00012661],
        [ 0.00014041],
        [-0.00014739],
        [-0.00013503]])

In [39]:
np.exp(V)

matrix([[0.48954166],
        [0.48954166],
        [5.29449005],
        [5.29449005]])

In [24]:
gamma_dict[2].T @ V

matrix([[ 0.19735342],
        [ 0.17854388],
        [-0.11362521],
        [-0.12096211]])

In [ ]:
q_dict=dict()
q_dict[1]=gamma_dict[0]/gamma_dict[1]

In [15]:
theta

array([0.09423409, 0.09423409, 0.09423409, 0.09423409])

In [18]:
dist['x']

array([0.35, 0.35, 0.15, 0.15])

In [19]:
dist['t_x']

array([0.25, 0.25, 0.25, 0.25])

In [16]:
sum(gamma)

array([0.12552459, 0.12552459, 0.20091186, 0.20091186])

In [ ]:

def algorithms(reg,m,n,g,f,C):
    K=np.exp(-C/C.max()/reg)
    interations=100
    trial=10 #check the optimiality every trial. 
    u=np.ones((n,1))
    u_record=np.ones((n,1))
    for tri in range(trial):
        if sum(abs(u_record-u))<10**(-10):
            for i in range(1,interations):
                v=g/np.dot(K.T,u)
                u=f/np.dot(K,v)
            u_record=u
        else:
            break
    return np.dot(np.diag(u.reshape((1,-1))[0]),np.dot(K,np.diag(v.reshape((1,-1))[0])))

def assess(m,n,g,f,C,output):
    print('sum of violation of f:',sum(abs(np.sum(output,1)-f.reshape(n))))
    print('sum of violation of g:',sum(abs(np.sum(output,0)-g.reshape(m))))
    print('total cost:',sum(sum(output*C)))
    print('entropy:',sum(sum(-output*np.log(output+0.1**3))))
    print('============================================')

def plots(x_range,g,f,output):
    fig = plt.figure(figsize=(3,3))
    gs = fig.add_gridspec(2, 2, width_ratios=(4,1), height_ratios=(1,4),left=0.1,right=0.9,bottom=0.1, top=0.9,wspace=0,hspace=0)
    # Create the Axes.
    ax = fig.add_subplot(gs[1, 0])
    ax.pcolormesh(x_range, x_range, output, cmap='Blues')
    ax.set_xlabel(r'$\mathbb{X}$',fontsize=14)
    ax.set_ylabel(r'$\tilde{\mathbb{X}}$',rotation='horizontal',fontsize=14)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax_histx = fig.add_subplot(gs[0, 0], sharex=ax) 
    ax_histy = fig.add_subplot(gs[1, 1], sharey=ax)
    #ax_histx.set_title(r'$Pr[x]$',rotation='horizontal')
    #ax_histy.set_title(r'$Pr[\tilde{x}]$')
    ax_histx.tick_params(axis="x", labelbottom=False)
    ax_histx.tick_params(axis="y", labelleft=False)
    ax_histy.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis="y", labelleft=False)
    ax_histx.plot(x_range,g,color='tab:blue')
    ax_histy.plot(f,x_range,color='tab:green') 
    return fig

def OT(reg,x_range,g,f):
    m=len(g)
    n=len(f)
    C=c_generate(x_range)
    output=algorithms(reg,m,n,g.reshape(m,-1),f.reshape(n,-1),C)
    assess(m,n,g,f,C,output)
    plots(x_range,g,f,output)
    return outputa